In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:

from sklearn.model_selection import train_test_split # to split the data into two parts
# from sklearn.cross_validation import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score


In [4]:
!pip install keras

  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)


In [6]:
!pip install tensorflow

  Using cached tensorflow-2.7.0-cp39-cp39-macosx_10_11_x86_64.whl (207.1 MB)
  Using cached libclang-12.0.0-py2.py3-none-macosx_10_9_x86_64.whl (12.2 MB)
  Using cached h5py-3.5.0-cp39-cp39-macosx_10_9_x86_64.whl (3.0 MB)
  Using cached grpcio-1.41.1-cp39-cp39-macosx_10_10_x86_64.whl (3.9 MB)
  Using cached wrapt-1.13.3-cp39-cp39-macosx_10_9_x86_64.whl (33 kB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached tensorflow_io_gcs_filesystem-0.22.0-cp39-cp39-macosx_10_14_x86_64.whl (1.6 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached wheel-0.37.0-py2.py3-none-any.whl (35 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3

In [9]:

import keras
from keras.layers import Dense
from keras.models import Sequential
#from keras.utils import to_categorical
#from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout


In [ ]:


import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")

%matplotlib inline

In [10]:
import mlflow

from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open(".mlflow_uri").read().strip()

"""
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

or parameters of the model (fit_intercept for Linear Regression model)
mlflow_params = {
      "altitude_low_meters_mean": altitude_low_meters_mean,
      "altitude_high_meters_mean": altitude_high_meters_mean,
      "altitude_mean_log_mean": altitude_mean_log_mean,
      "fit_intercept": True,
  }


# logging params to mlflow
#mlflow.log_params(params)
# setting tags
mlflow.set_tag("model", "XGBoost")
mlflow.set_tag("features", "imbalance price")
# logging metrics
# mlflow.log_metric("train-" + "RMSE", rmse_train)
# mlflow.log_metric("test-" + "RMSE", rmse_test)
# mlflow.log_metric("train-" + "R2", rsquared_train)
# mlflow.log_metric("test-" + "R2", rsquared_test)
# end run
# mlflow.end_run()
"""

'\n# setting the MLFlow connection and experiment\nmlflow.set_tracking_uri(TRACKING_URI)\nmlflow.set_experiment(EXPERIMENT_NAME)\nmlflow.start_run()\nrun = mlflow.active_run()\n\nor parameters of the model (fit_intercept for Linear Regression model)\nmlflow_params = {\n      "altitude_low_meters_mean": altitude_low_meters_mean,\n      "altitude_high_meters_mean": altitude_high_meters_mean,\n      "altitude_mean_log_mean": altitude_mean_log_mean,\n      "fit_intercept": True,\n  }\n\n\n# logging params to mlflow\n#mlflow.log_params(params)\n# setting tags\nmlflow.set_tag("model", "XGBoost")\nmlflow.set_tag("features", "imbalance price")\n# logging metrics\n# mlflow.log_metric("train-" + "RMSE", rmse_train)\n# mlflow.log_metric("test-" + "RMSE", rmse_test)\n# mlflow.log_metric("train-" + "R2", rsquared_train)\n# mlflow.log_metric("test-" + "R2", rsquared_test)\n# end run\n# mlflow.end_run()\n'

In [11]:
train = pd.read_csv('../data/train.csv')
train.head()

,datetime,1
0,2013-12-31 23:00:00,917.118
1,2014-01-01 00:00:00,1264.266
2,2014-01-01 01:00:00,746.810
3,2014-01-01 02:00:00,-15.512
4,2014-01-01 03:00:00,-36.904


In [13]:
train = train.rename(columns={'1':'imb_price'})
train.head()

,datetime,imb_price
0,2013-12-31 23:00:00,917.118
1,2014-01-01 00:00:00,1264.266
2,2014-01-01 01:00:00,746.810
3,2014-01-01 02:00:00,-15.512
4,2014-01-01 03:00:00,-36.904


In [12]:
test = pd.read_csv('../data/test.csv')
test.head()

,datetime,1
0,2019-03-05 06:00:00,-349.8990
1,2019-03-05 07:00:00,372.9630
2,2019-03-05 08:00:00,371.4125
3,2019-03-05 09:00:00,817.3485
4,2019-03-05 10:00:00,638.5965


In [14]:
test = test.rename(columns={'1':'imb_price'})
test.head()

,datetime,imb_price
0,2019-03-05 06:00:00,-349.8990
1,2019-03-05 07:00:00,372.9630
2,2019-03-05 08:00:00,371.4125
3,2019-03-05 09:00:00,817.3485
4,2019-03-05 10:00:00,638.5965


In [17]:
# shift function for one-lagged series for training data set

train['shift1'] = train['imb_price'].shift(-1)
train.tail()

,datetime,imb_price,shift1
45338,2019-03-05 01:00:00,147.278,-28.272
45339,2019-03-05 02:00:00,-28.272,-201.291
45340,2019-03-05 03:00:00,-201.291,83.710
45341,2019-03-05 04:00:00,83.710,-102.683
45342,2019-03-05 05:00:00,-102.683,NaN


In [16]:
# shift function for one-lagged series for test data set

test['shift1'] = test['imb_price'].shift(-1)
test.tail()

,datetime,imb_price,shift1
19428,2021-05-22 18:00:00,-1.051667,14.628167
19429,2021-05-22 19:00:00,14.628167,30.308000
19430,2021-05-22 20:00:00,30.308000,164.239000
19431,2021-05-22 21:00:00,164.239000,298.170000
19432,2021-05-22 22:00:00,298.170000,NaN


In [18]:
# removing the last line in the training data set
train = train.dropna()
train.tail()

,datetime,imb_price,shift1
45337,2019-03-05 00:00:00,-328.8805,147.278
45338,2019-03-05 01:00:00,147.2780,-28.272
45339,2019-03-05 02:00:00,-28.2720,-201.291
45340,2019-03-05 03:00:00,-201.2910,83.710
45341,2019-03-05 04:00:00,83.7100,-102.683


In [19]:
# removing the last line in the test data set
test = test.dropna()
test.tail()

,datetime,imb_price,shift1
19427,2021-05-22 17:00:00,-16.731500,-1.051667
19428,2021-05-22 18:00:00,-1.051667,14.628167
19429,2021-05-22 19:00:00,14.628167,30.308000
19430,2021-05-22 20:00:00,30.308000,164.239000
19431,2021-05-22 21:00:00,164.239000,298.170000


In [20]:
# removing datetime column time in train data set in order to convert it into a non-time series problem
train = train.drop('datetime', axis=1)
train.head()

,imb_price,shift1
0,917.118,1264.266
1,1264.266,746.810
2,746.810,-15.512
3,-15.512,-36.904
4,-36.904,360.392


In [21]:
# removing datetime column time in test data set in order to convert it into a non-time series problem
test = test.drop('datetime', axis=1)
test.head()

,imb_price,shift1
0,-349.8990,372.9630
1,372.9630,371.4125
2,371.4125,817.3485
3,817.3485,638.5965
4,638.5965,-206.9860


In [22]:
scaler = MinMaxScaler()

In [23]:
train = scaler.fit_transform(train)

In [24]:
test = scaler.transform(test)

In [25]:
train.shape

(45342, 2)

In [29]:
train_X, train_y = train[:,[0]], train[:,[1]]

In [32]:
test_X, test_y = test[:,[0]], test[:,[1]]

In [33]:
train_X.shape

(45342, 1)

In [45]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 

(45342, 1, 1) (45342, 1) (19432, 1, 1) (19432, 1)


In [47]:
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
#    model.add(LSTM(70))
#    model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [48]:
# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=70, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/20
648/648 - 4s - loss: 0.0114 - val_loss: 0.0034 - 4s/epoch - 6ms/step
Epoch 2/20
648/648 - 2s - loss: 0.0036 - val_loss: 0.0034 - 2s/epoch - 2ms/step
Epoch 3/20
648/648 - 2s - loss: 0.0035 - val_loss: 0.0035 - 2s/epoch - 3ms/step
Epoch 4/20
648/648 - 2s - loss: 0.0034 - val_loss: 0.0036 - 2s/epoch - 3ms/step
Epoch 5/20
648/648 - 2s - loss: 0.0034 - val_loss: 0.0037 - 2s/epoch - 3ms/step
Epoch 6/20
648/648 - 2s - loss: 0.0033 - val_loss: 0.0038 - 2s/epoch - 3ms/step
Epoch 7/20
648/648 - 2s - loss: 0.0033 - val_loss: 0.0038 - 2s/epoch - 3ms/step
Epoch 8/20
648/648 - 2s - loss: 0.0032 - val_loss: 0.0037 - 2s/epoch - 3ms/step
Epoch 9/20
648/648 - 2s - loss: 0.0032 - val_loss: 0.0037 - 2s/epoch - 3ms/step
Epoch 10/20
648/648 - 2s - loss: 0.0031 - val_loss: 0.0037 - 2s/epoch - 3ms/step
Epoch 11/20
648/648 - 2s - loss: 0.0031 - val_loss: 0.0036 - 2s/epoch - 3ms/step
Epoch 12/20
648/648 - 2s - loss: 0.0031 - val_loss: 0.0035 - 2s/epoch - 2ms/step
Epoch 13/20
648/648 - 2s - loss: 0.00